In [1]:
import os
from uuid import uuid4
from dotenv import load_dotenv
from langchain_community.tools import YouTubeSearchTool
from langchain_openai import ChatOpenAI as Chat
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph

load_dotenv()

True

In [2]:
openai_api_key = os.getenv("OPENAI_API_KEY")
model = os.getenv("OPENAI_MODEL", "gpt-4o")
temperature = float(os.getenv("OPENAI_TEMPERATURE", 0))

NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
llm = Chat(
    openai_api_key=openai_api_key,
    model=model,
    temperature=temperature
)

In [4]:
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [5]:
# Generate a unique session ID
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")


def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

Session ID: 1f6e22ea-ae43-4956-8af5-ce845706e041


In [6]:
# Initialize YouTube search tool
youtube = YouTubeSearchTool()

def call_trailer_search(input):
    input = input.replace(",", " ")
    return youtube.run(input)

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie expert. You find movies from a genre or plot."),
    ("human", "{input}"),
])

In [8]:
movie_chat_chain = prompt | llm | StrOutputParser()

In [9]:
# Define the tools for the agent
tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat_chain.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
]

In [10]:
# Pull the agent prompt from the hub
agent_prompt = hub.pull("hwchase17/react-chat")

# Create the agent and agent executor
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [11]:
# Create the chat agent with message history
chat_agent_chain = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
    handle_parsing_errors=True,
)

In [ ]:
while True:
    q = input("> ")
    response = chat_agent_chain.invoke(
        {"input": q},
        {"configurable": {"session_id": SESSION_ID}},
        )
    print(response["output"])

Hello! How can I assist you today?
Here are two links to the trailers for "Avengers: Age of Ultron":

1. [Trailer 1](https://www.youtube.com/watch?v=tmeOjFno6Do&pp=ygUnYXZlbmdlcnMgYWdlIG9mIHVsdHJvbiB0cmFpbGVyCmBgYAoKYGBg)
2. [Trailer 2](https://www.youtube.com/watch?v=JAUoeqvedMo&pp=ygUnYXZlbmdlcnMgYWdlIG9mIHVsdHJvbiB0cmFpbGVyCmBgYAoKYGBg)

Enjoy watching!
